In [3]:
import heapq, os

class BinaryTreeNode:
    
    def __init__(self, value, freq):
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None
        
    # OPERATOR OVERLOADING FOR HEAP COMPARISIONS.
    def __lt__(self, other):
        return self.freq < other.freq
    
    def __eq__(self, other):
        return self.freq == other.freq

class HuffmanCoding:
    
    def __init__(self, path):
        self.path = path
        self.__heap = []
        self.__codes = {}
        self.__reverse_codes = {}
        
    def __make_frequency_dict(self, text):
        freq_dict = {}
        for char in text:
            freq_dict[char] = freq_dict.get(char, 0) + 1
        return freq_dict
    
    def __buildHeap(self, freq_dict):
        # We are inserting a Binary Tree Node in the heap and comparision in heap
        # will be based on the frequency present in that node. 
        for key,freq in freq_dict.items():
            binary_tree_node = BinaryTreeNode(key, freq)
            heapq.heappush(self.__heap, binary_tree_node)
    
    def __buildTree(self):
        while len(self.__heap) > 1:
            # Taking out Two Minimum nodes
            binary_tree_node_1 = heapq.heappop(self.__heap)
            binary_tree_node_2 = heapq.heappop(self.__heap)
            # Inserting newNode with no value that stores sum of two frequencies
            freqSum = binary_tree_node_1.freq + binary_tree_node_2.freq
            newNode = BinaryTreeNode(None, freqSum)
            # setting left and right children of newNode
            newNode.left = binary_tree_node_1
            newNode.right = binary_tree_node_2
            # pushing newNode into the heap
            heapq.heappush(self.__heap, newNode)
        # At this point there is one node left in the heap, and that is the root of the Binary Tree constructed
        return self.__heap.pop()
    
    def __buildCodesRecursiveHelper(self, root, curr_bits):
        # Base Case
        if root is None: return
        # if the node has non None value that means it must have a char ending and is a leaf node. 
        if root.value is not None:
            char = root.value
            self.__codes[char] = curr_bits
            self.__reverse_codes[curr_bits] = char
        
        self.__buildCodesRecursiveHelper(root.left, curr_bits + '0')
        self.__buildCodesRecursiveHelper(root.right, curr_bits + '1')
    
    def __buildCodes(self, root):
        self.__buildCodesRecursiveHelper(root, "")
        
    def __getEncodedText(self, text):
        encoded_text = ''
        for char in text:
            encoded_text += self.__codes[char]
        return encoded_text
    
    def __getPaddedEncodedText(self, encoded_text):
        # Add reqd number of zeros in end First
        paddingLength = 8 - (len(encoded_text) % 8)
        for i in range(paddingLength):
            encoded_text += '0'
        
        # Add first 8 bits of string as the paddingLength
        padded_info = "{0:08b}".format(paddingLength)
        padded_encoded_text = padded_info + encoded_text
        return padded_encoded_text
            
    def __getBytesArray(self, padded_encoded_text):
        # for every 8 bits we convert into and integer decimal, and store it in array
        array = []
        for i in range(0,len(padded_encoded_text),8):
            byte = padded_encoded_text[i : i + 8]
            array.append(int(byte, 2)) # this means convert this string into integer taking base 2 ; because it is in binary representaion
        return array
        
    def compress(self):
        # Get file from the path provided by user
        file_name, file_extension = os.path.splitext(self.path)
        output_path = file_name + '.bin'
        
        # Read text from the file; r+ denotes reading and wb denotes Write  as Binary file
        with open(self.path, 'r+') as file, open(output_path, 'wb') as output:
            text = file.read()
            text = text.rstrip() # remove trailing spaces
        
            # Make frequency-dictionary using that text
            freq_dict = self.__make_frequency_dict(text)
            
            # Construct the heap from the frequency-dictionary
            self.__buildHeap(freq_dict)

            # Construct the binary tree fomm the heap
            root = self.__buildTree()
            
            # Construct the codes from Binary Tree
            self.__buildCodes(root)

            # Creating the encoded text using the codes
            encoded_text = self.__getEncodedText(text)
            
            # PADDING : for a byte we take 8 bits each time, but there can be a case 
            # such that in end less than 8 bits are left, for them pad the extra reqd bits with 0's
            # And for later instance to know that how much 0's we have added, store that number as a 8 bit representaion
            # as the first 8 bits of the encoded_text

            # Pad this Encoded Text
            padded_encoded_text = self.__getPaddedEncodedText(encoded_text)
            
            # Put this encoded text into a binary file.
            bytes_array = self.__getBytesArray(padded_encoded_text)
        
            # Return this binary file as output
            final_bytes = bytes(bytes_array)
            
            output.write(final_bytes)
            
        print('Compressed!')
        return output_path
             
    def __removePadding(self, text):
        padded_info = text[:8]
        extra_padding = int(padded_info, 2)
        
        text = text[8:]
        text_after_padding_removed = text[:(-1)*extra_padding]
        return text_after_padding_removed
    
    def __decodeText(self, text):
        decoded_text = ""
        current_bits = ""
        for bit in text:
            current_bits += bit
            if current_bits in self.__reverse_codes:
                character = self.__reverse_codes[current_bits]
                decoded_text += character
                current_bits = ""
        return decoded_text
    
    def decompress(self, input_path):
        file_name, file_extension = os.path.splitext(self.path)
        output_path = file_name + 'decompressed' +  '.txt'
        with open(input_path, 'rb') as file, open(output_path, 'w') as output: # reading a Binary!
            bit_string = "" # bit_string  will be generated to padded_encoded_text
            byte = file.read(1) # mentioning 1 will help in reading each code one by one.
            while byte:  # Till byte is not None
                byte = ord(byte) # ciinverting that byte into corresponsing integer
                bits = bin(byte)[2:].rjust(8, '0') # convert that integer into binary form and then making it equal to 8 bits 
                bit_string += bits
                byte = file.read(1)
                
            # Remove Padding
            actual_encoded_text = self.__removePadding(bit_string)
            
            # Decode the encoded_text into the characters.
            decompressed_text = self.__decodeText(actual_encoded_text)
            
            output.write(decompressed_text)
        print('Decompressed!')
        return

In [4]:
path = "/Users/Deepanshu Aggarwal/Desktop/sample.txt"
h = HuffmanCoding(path)
output_path = h.compress()
h.decompress(output_path)

Compressed!
Decompressed!
